In [5]:
import pandas as pd
from ipywidgets import interact, widgets, interactive_output
from IPython.display import display, HTML

# load the data
url = "https://raw.githubusercontent.com/OHDSI/GIS/gis_vocab_package_dev/vocabularies/gis_vocab_v2/GIS-prestage-for-testing.csv"
data = pd.read_csv(url)

# define search columns
SEARCH_COLUMNS = ['concept_code_1', 'concept_name_1', 'concept_class_id_1', 'vocabulary_id_1', 
                  'concept_code_2', 'concept_name_2', 'concept_class_id_2', 'vocabulary_id_2'] 

def search_and_filter(column_for_search, search_query):
    result = data

    # filter by search query
    if search_query:
        # convert the column to string
        result[column_for_search] = result[column_for_search].astype(str)
        
        # exact matches first
        exact_matches = result[result[column_for_search].str.lower() == search_query.lower()]
        
        # all matches containing the search query
        all_matches = result[result[column_for_search].str.lower().str.contains(search_query.lower(), na=False)]
        
        # exclude exact matches from all matches
        other_matches = all_matches[~all_matches.index.isin(exact_matches.index)]
        
        # sort other matches by length
        other_matches = other_matches.assign(length=other_matches[column_for_search].str.len()).sort_values(by='length').drop(columns='length')

        # concatenate exact matches and other matches
        result = pd.concat([exact_matches, other_matches])

    display(result)

# widgets for input
search_bar = widgets.Text(placeholder='Search', description='Search')
column_for_search_input = widgets.Dropdown(
    options=SEARCH_COLUMNS,
    description='Column'
)

# interact function
interact(search_and_filter, 
         column_for_search=column_for_search_input, 
         search_query=search_bar)

interactive(children=(Dropdown(description='Column', options=('concept_code_1', 'concept_name_1', 'concept_cla…

<function __main__.search_and_filter(column_for_search, search_query)>

In [6]:
# create slider and text widgets for row index to see all inforamtion about a term
row_slider = widgets.IntSlider(min=0, max=len(data)-1, step=1, description='Row Index:')
row_text = widgets.IntText(description='Row Index:')
widgets.jslink((row_slider, 'value'), (row_text, 'value'))

def browse_data(row_index):
    row = data.iloc[row_index]
    display(HTML(f"<strong>Row index:</strong> {row_index}"))
    display(HTML(row.to_frame().to_html()))

# create a combined widget display
ui = widgets.HBox([row_slider, row_text])
out = interactive_output(browse_data, {'row_index': row_slider})

display(ui, out)

Output()